In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define constants
IMG_SIZE = 128
NUM_CLASSES = 5
BATCH_SIZE = 32
NUM_EPOCHS = 20

# Define class mapping
class_map = {
    'line': 0,
    'dot_line': 1,
    'hbar_categorical': 2,
    'vbar_categorical': 3,
    'pie': 4
}

# Load data
data_dir = '/content/drive/My Drive/Colab Notebooks/Machine Learning/Assignment 2/Problem 3/charts/'
labels = pd.read_csv(os.path.join(data_dir, 'train_val.csv'))
labels['type'] = labels['type'].map(class_map)
X = []
y = []
for i, row in labels.iterrows():
    img = cv2.imread(os.path.join(data_dir, 'train_val', str(row['image_index'] )+".png"))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    X.append(img)
    y.append(row['type'])
X = np.array(X)
y = to_categorical(y, NUM_CLASSES)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))


Epoch 1/20
25/25 [==============================] - 26s 1s/step - loss: 184.7742 - accuracy: 0.4737 - val_loss: 0.8810 - val_accuracy: 0.7500
Epoch 2/20
25/25 [==============================] - 25s 986ms/step - loss: 1.1619 - accuracy: 0.6975 - val_loss: 0.6281 - val_accuracy: 0.8300
Epoch 3/20
25/25 [==============================] - 27s 1s/step - loss: 0.5831 - accuracy: 0.8125 - val_loss: 0.1749 - val_accuracy: 0.9300
Epoch 4/20
25/25 [==============================] - 25s 1s/step - loss: 0.2350 - accuracy: 0.9125 - val_loss: 0.1183 - val_accuracy: 0.9450
Epoch 5/20
25/25 [==============================] - 25s 988ms/step - loss: 0.1902 - accuracy: 0.9625 - val_loss: 0.0390 - val_accuracy: 0.9900
Epoch 6/20
25/25 [==============================] - 24s 941ms/step - loss: 0.0616 - accuracy: 0.9787 - val_loss: 0.0522 - val_accuracy: 0.9800
Epoch 7/20
25/25 [==============================] - 25s 1s/step - loss: 0.0542 - accuracy: 0.9862 - val_loss: 0.0537 - val_accuracy: 0.9850
Epoch 8/2

In [3]:
# Evaluate model on test data
test_labels = pd.read_csv(os.path.join(data_dir, 'test.csv'))
test_labels['type'] = test_labels['type'].map(class_map)
X_test = []
y_test = []
for i, row in test_labels.iterrows():
    img = cv2.imread(os.path.join(data_dir, 'test', str(row['image_index'] )+".png"))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    X_test.append(img)
    y_test.append(row['type'])
X_test = np.array(X_test)
y_test = to_categorical(y_test, NUM_CLASSES)
loss, acc = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {acc:.4f}')

2/2 [==============================] - 0s 93ms/step - loss: 0.6459 - accuracy: 0.9130
Test loss: 0.6459
Test accuracy: 0.9130
